# UFCNN, San Juan, total cases is only feature

In [3]:
import scipy as sp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels as stats
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
import theano 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras import initializers, regularizers
from keras.layers import Dense, Activation, Dropout, Convolution1D, Merge
%matplotlib inline
import myutil_ufcnn as myutil
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [4]:
import importlib
_ = importlib.reload(myutil)

## Read data.  Will use total_cases as single input feature.

In [24]:
all_input = myutil.get_indexed_dataset('data/dengue_labels_train.csv')
all_input.head()

,city,year,weekofyear,total_cases
yearweekofyear,,,,
199018,sj,1990,18,4
199019,sj,1990,19,5
199020,sj,1990,20,4
199021,sj,1990,21,3
199022,sj,1990,22,6


### Get and shape San Juan data

In [25]:
input_sj = all_input[all_input['city']=='sj']['total_cases'].values
input_sj = input_sj.reshape(input_sj.shape[0],1,1)
input_sj.shape

(936, 1, 1)

### Get and shape Iquitos data

In [27]:
input_iq = all_input[all_input['city']=='iq']['total_cases'].values
input_iq = input_iq.reshape(input_iq.shape[0],1,1)
input_iq.shape

(520, 1, 1)

In [8]:
if 1==2:
    for city in best_result.keys():
        print('Best result for {}'.format(city))
        print('Best dataset: {}'.format(best_result[city][1]))
        print('\t\tFinal loss train: {}'.format(best_result[city][2]))
        print('\t\tFinal loss valid: {}'.format(best_result[city][3]))
        print('\t\tVariance score: %.2f' % best_result[city][4])
        plt.plot(best_result[city][0]['loss'], label='train')
        plt.plot(best_result[city][0]['val_loss'], label='test')
        plt.legend()
        plt.show()

In [9]:
def print_nodes_shapes(model):
    for layer in model.layers:
        for attrib in layer:
            print(attrib)


In [12]:
def fix_predictions(raw_predictions):
    raw_predictions[ raw_predictions < 0] = 0
    raw_predictions = np.around(raw_predictions).astype('int')
    return raw_predictions
    
def run_ufcnn(X_train_, X_valid_, y_train_, y_valid_, model_parms):
    
    sequence_length=X_train_.shape[0]
    features=1
    nb_filter=int(sequence_length/4)     # Number of convolution kernels to use (dimensionality of the output)
    filter_length=2                      # The extension (spatial or temporal) of each filter
    output_dim=1
    optimizer='adagrad'
    loss='mse'
    input_shape=(1,1)
    
    X_train_ = X_train_[:,-1:].reshape(X_train_.shape[0], 1, 1)
    X_valid_ = X_valid_[:,-1:].reshape(X_valid_.shape[0], 1, 1)
    
    #print(X_train.shape)

    # build graph
    H1 = Sequential()
    H1.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',
                         activation='relu', input_shape=input_shape))
    
    H2 = Sequential()
    H2.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features)))
    H2.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features)))
    
    H3 = Sequential()
    H3.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features)))
    H3.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features)))
    H3.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features)))
    
    G3 = Sequential()
    G3.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features)))
    G3.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features)))
    G3.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features)))
    G3.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features)))  
    
    merged = Merge([H2, G3], mode='sum')
    G2 = Sequential()
    G2.add(merged)
    G2.add(Activation('relu'))
    
    merged = Merge([H1, G2], mode='sum')
    G1 = Sequential()
    G1.add(merged)
    G1.add(Activation('relu'))
    G1.add(Convolution1D(nb_filter=output_dim, filter_length=filter_length, border_mode='same',\
                         activation='relu', input_shape=(sequence_length, features))) 
    
    history = regr.fit(X_train_, y_train_, epochs=20, batch_size=2,\
                       validation_data=(X_valid_, y_valid_), verbose=0, shuffle=False)
    
    y_hat = fix_predictions(G1.predict(X_train_))
    y_pred = fix_predictions(G1.predict(X_valid_))

    loss_train = history.history['loss'][-1]
    loss_valid = history.history['val_loss'][-1]
    r2 = r2_score(y_valid_, y_pred)
    score = r2/loss_valid
    
    return G1, score, r2, loss_train, loss_valid, history.history
    #       0      1     2      3          4             5

    
def score_dataset(X_train, y_train):
    
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.33, random_state=42)
    
    best_score = 0
    best_results = (0, 0, 0, 0, 0, 0)
    
    params = ['dummy'] 
    for parms in params:        
        model, this_score, r2, loss_train, loss_valid, _ = run_ufcnn(X_train, X_valid, y_train, y_valid, parms)
        # only check scores if loss_train < loss_valid
        if (loss_train < loss_valid) and (this_score > best_score):
            best_results = params, model, this_score, r2, loss_train, loss_valid
            #print('Database: {} / Train Loss: {} / Valid Loss {} / R2: {}'.format(id, loss_train, loss_valid, r2))  
    return best_results
            

In [11]:
best_score = 0
best_results = ()

for dataset_name in dataset.keys():
    print(dataset_name)
    # get partitions for this version of the dataset
    X_train, y_train = dataset[dataset_name]['X_train'], dataset[dataset_name]['y_train']
    # get model and best score data for this dataset
    params, model, this_score, r2, loss_train, loss_valid = score_dataset(X_train, y_train)
    # save best score so far
    if this_score > best_score:
        best_score = dataset_name, params, model, this_score, r2, loss_train, loss_valid
        print('Database: {} / Train Loss: {} / Valid Loss {} / R2: {}'.format(dataset_name, loss_train, loss_valid, r2))

sj_8_raw
(924, 8)


ValueError: Only layers of same output shape can be merged using sum mode. Layer shapes: [(None, 1, 154), (None, 619, 154)]